In [ ]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/keras-idiomatic-programmer/blob/master/workshops/Idiomatic%20Programmer%20-%20handbook%203%20-%20Codelab%201.ipynb">
<img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>

# Idiomatic Programmer Code Labs

## Code Labs #1 - Get Familiar with Hyperparameters

## Prerequistes:

    1. Familiar with Python
    2. Completed Handbook 3/Part 10: Training Preparation and Hyperparameters

## Objectives:

    1. Hand setting Epochs and Mini-Batches
    2. Use ImageDataGenerator for batch generation.
    3. Finding good learning rate.

## Epochs and Mini-Batches

In this section, we will hand-roll our own code (vs. builtin feeders) to feed the training data for training. We will need to handle the following:

    1. Set a mini-batch size (128) and calculate how many batches will be in the training data.
    1. Set the number of epochs (number of times we pass the full training data for training)
    2. Randomly shuffle the training data on each epoch.
    3. Iterate through the training data on batch at a time.
    
You fill in the blanks (replace the ??), make sure it passes the Python interpreter.

In [ ]:
from keras.datasets import cifar10
import random

# Let's use the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# We will use a mini-batch size of 128
batch_size = 128

# Calculate the total number of mini-batches in an epoch
# HINT: It has something to do with the (mini) batch size
batches = len(x_train) // ??

# Let's use a seed so we can randomly shuffle both the pixel data and labels the same shuffle.
seed = 101

# Let's do 5 passes (epochs) over the dataset
epochs = 5
for epoch in range(epochs):
    # Shuffle the dataset at the beginning of each epoch
    # HINT: We have to shuffle the image data and labels from the training data
    random.seed(seed)
    random.shuffle(??)
    random.seed(seed)
    random.shuffle(??)
    # Set a new seed for the next shuffle
    seed += random.randint(0, 100)
    
    # Iterate (sequential) through the shuffled training data, one batch at a time.
    for batch in range(batches):
        # Get the next batch of data
        # HINT: if the begin of the batch is at location X, then the end is X + batc
        x_batch = x_train[batch * batch_size:(batch+??) * batch_size]
        y_batch = y_train[batch * batch_size:(batch+??) * batch_size]
        print("Epoch", epoch+1, "Batch", batch+1)
        
print("Done - the last line above this should be: Epoch 5, Batch 390")
    

## ImageDataGenerator and Batch Generation

In this section, we will use the **Keras** ImageDataGenerator to automatically generate out mini-batches (vs. hand generating them), and shuffling the training data on each epoch.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Let's use the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# We will use a mini-batch size of 128
batch_size = 128

# Calculate the total number of mini-batches in an epoch
batches = len(x_train) // batch_size

# instantiate an Image Data generator object
# HINT: Image Data generator is a big giveaway
datagen = ??()

# Let's do 5 passes (epochs) over the dataset
epochs = 5
for epoch in range(epochs):
    
    # Use generator to create batches
    # HINT: The method is about flowing data from in-memory (vs. on-disk)
    batch = 0
    for x_batch, y_batch in datagen.??(x_train, y_train, batch_size=batch_size, shuffle=True):
        batch += 1 # Keep track of the number of batches so far.
        print("Epoch", epoch+1, "Batch", batch)
        
        # At the end of the training data, let's loop around for the next epoch.
        if batch == batches:
            break
            
print("Done - the last line above this should be: Epoch 5, Batch 390")

## Learning Rate

Let's show how to do short epochs to get a feel on what might be the right learning rate for your training.

In [ ]:
from keras import Sequential, optimizers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
import numpy as np

# Let's use the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel data
x_train = (x_train / 255.0).astype(np.float32)
x_test  = (x_test  / 255.0).astype(np.float32)

# One-hot encode the labels
y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)


def convNet(input_shape, nclasses):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(nclasses, activation='softmax'))
    return model

# Create a simple CNN and set learning rate very high (0.1))
# HINT: how would you abbreviate learning rate?
model = convNet((32, 32, 3), 10)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(??=0.1),
              metrics=['accuracy'])

# Let's take a fraction of the training data to test the learning rate (2%)
x_tmp = x_train[0:1000]
y_tmp = y_train[0:1000]

# Let's run 3 epochs at learning rate = 0.1
model.fit(x_tmp, y_tmp, epochs=3, batch_size=32, verbose=1)

Argh, it's horrible. The loss on the first epoch is high (14.0+) and then never goes down - like it's stuck.

Hum, okay now you experiment with different learning rates to find one where the loss goes down rapidly and a steady increase in accuracy.

In [ ]:
model = convNet((32, 32, 3), 10)
# Pick your own learning rate until the results are good.
# HINT: It's going to be a lot smaller than 0.1
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=??),
              metrics=['accuracy'])

# Let's run 3 epochs at your learning rate
model.fit(x_tmp, y_tmp, epochs=3, batch_size=32, verbose=1)

## End of Code Lab